In [1]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
trainData = pd.read_csv("train.csv")
testData = pd.read_csv("test.csv")

In [3]:
trainData

,POS_0_9_CP,POS_10_19_CP,POS_20_29_CP,POS_30_39_CP,POS_40_49_CP,POS_50_59_CP,POS_60_69_CP,POS_70_79_CP,POS_80_89_CP,POS_90_CP,DTH_CUM_CP
0,24,51,44,51,60,75,72,61,27,0,3
1,22,31,214,177,62,46,24,9,0,0,2
2,84,126,239,194,159,171,125,74,24,8,9
3,65,76,80,110,82,81,55,49,56,34,7
4,99,107,339,287,139,147,110,71,37,0,12
...,...,...,...,...,...,...,...,...,...,...,...
1039,86,201,206,202,240,317,225,138,48,0,7
1040,80,102,125,124,104,71,70,36,6,0,6
1041,268,358,277,415,355,332,215,107,47,7,5
1042,81,116,90,145,118,113,70,36,9,0,2


In [4]:
testData

,POS_0_9_CP,POS_10_19_CP,POS_20_29_CP,POS_30_39_CP,POS_40_49_CP,POS_50_59_CP,POS_60_69_CP,POS_70_79_CP,POS_80_89_CP,POS_90_CP,DTH_CUM_CP
0,53,110,80,91,103,134,131,83,38,0,9
1,48,104,102,133,154,202,118,104,72,67,26
2,45,67,81,99,66,74,38,18,11,0,2
3,61,77,121,107,65,55,35,15,8,0,7
4,123,137,177,163,193,147,115,86,29,9,9
...,...,...,...,...,...,...,...,...,...,...,...
343,115,195,236,328,233,225,176,79,28,6,24
344,79,137,250,218,164,205,146,51,28,0,14
345,204,215,383,341,234,224,184,84,23,8,10
346,50,116,100,90,113,139,115,60,27,9,11


In [5]:
np_train = np.array(trainData)
np_test = np.array(testData)

In [6]:
train = torch.from_numpy(np_train)   # numpy to tensor
train = train.to(torch.float64)
test = torch.from_numpy(np_test)     # numpy to tensor
test = test.to(torch.float64)

In [7]:
train

tensor([[ 24.,  51.,  44.,  ...,  27.,   0.,   3.],
        [ 22.,  31., 214.,  ...,   0.,   0.,   2.],
        [ 84., 126., 239.,  ...,  24.,   8.,   9.],
        ...,
        [268., 358., 277.,  ...,  47.,   7.,   5.],
        [ 81., 116.,  90.,  ...,   9.,   0.,   2.],
        [118., 156., 197.,  ...,   0.,   0.,   5.]], dtype=torch.float64)

In [8]:
trainX = train[:, :-1]
trainY = train[:, -1]
trainY = trainY.view(-1, 1)

In [9]:
testX = test[:, :-1]
testY = test[:, -1]
testY = testY.view(-1, 1)

In [10]:
#q1
byteSize = trainX.nelement() * trainX.element_size()
byteSize

83520

In [11]:
#q2
tempTrainX = trainX.to(torch.float16)
tempTrainX2 = tempTrainX.to(torch.float64)
diffMatrix = trainX - tempTrainX2
maxAbsDiff = float(torch.max(diffMatrix))
maxAbsDiff

0.0

In [12]:
#q3
torch.cuda.is_available()

False

In [13]:
coef = torch.tensor([
        [0.0040],
        [0.0040],
        [0.0040],
        [0.0040],
        [0.0040],
        [0.0040], # POS_50_59_CP
        [0.0300], # POS_60_69_CP
        [0.0300],
        [0.0300],
        [0.0300]
], dtype=trainX.dtype)
coef

tensor([[0.0040],
        [0.0040],
        [0.0040],
        [0.0040],
        [0.0040],
        [0.0040],
        [0.0300],
        [0.0300],
        [0.0300],
        [0.0300]], dtype=torch.float64)

In [14]:
#q4
predictedDeathsRowOne = testX[0] @ coef
resultQ4 = predictedDeathsRowOne.item()
resultQ4

9.844

In [15]:
#q5
predictedDeathsAll = testX @ coef
predictedDeathsAll
resultQ5 = torch.mean(predictedDeathsAll)
resultQ5.item()

12.073632183908048

In [16]:
#q6
def f(x):
    return x * x - 8 * x + 19

x = torch.tensor(0.0)
y = f(x)
float(y)

19.0

In [17]:
#q7
x = torch.tensor(0.0, requires_grad=True)
optimizer = torch.optim.SGD([x], lr=0.01) # minimizes by default

for epoch in range(1000):
    y = f(x)
    y.backward()
    optimizer.step()
    optimizer.zero_grad()

x.item()

3.9999940395355225

In [18]:
#q8
loss_fn = torch.nn.MSELoss()
coef = torch.zeros((10, 1), dtype=torch.float64, requires_grad=True)
predictions = trainX @ coef
mse = loss_fn(predictions, trainY)
mse.item()

197.8007662835249

In [19]:
torch.manual_seed(544)
ds = torch.utils.data.TensorDataset(trainX, trainY)
dl = torch.utils.data.DataLoader(ds, batch_size=50, shuffle=True)

In [20]:
loss_fn = torch.nn.MSELoss()
coef = torch.zeros((10, 1), dtype=torch.float64, requires_grad=True)
optimizer = torch.optim.SGD([coef], lr=0.000002)

for epoch in range(500):
    for batchX, batchY in dl:
        predictions = batchX @ coef
        loss = loss_fn(predictions, batchY)
        loss.backward()    # computes gradient, and adds it to coef. grad
        optimizer.step()
        optimizer.zero_grad()

In [21]:
#q9
loss_fn(trainX @ coef, trainY).item()

26.8113940147193

In [22]:
#q10
loss_fn(testX @ coef, testY).item()

29.05854692548551